# Simple Convolutional Neural Network

In [1]:
import numpy as np
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [6]:
# Create placeholder for input and output
# Placeholder because we use it for inserting data
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [2]:
# Start Interactive Session
sess = tf.InteractiveSession()

In [3]:
# Weight Initialization
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [4]:
# Convolution and Pooling Layer
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

### First Convolutional Layer

In [5]:
# Initialize Weight and Bias
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

In [7]:
# Reshape the input x, into 4d tensor
x_image = tf.reshape(x, [-1, 28, 28, 1])

In [8]:
# In the hidden layer, we have convolutional and pooling layer
# We convolve the input with weight, add to bias and apply ReLU function
# Then we apply max pool
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

### Second Convolutional Layer

In [9]:
# Initialize Weight and Bias
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

In [10]:
# Continuation from previous layer,
# We convolve the last output with weight, add to bias and apply ReLU function
# Then we apply max pool
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

### Dense Layer / Fully Connected Layer

In [11]:
# Fully connected layer, with the last output size
# We initialize the weight and bias first
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

In [13]:
# Then we flatten the output, and pass it to fully connected later
# And we also apply ReLU to it
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

### Dropout 

In [14]:
# Here we implement dropout
# TensorFlow's dropout automatically handle the scaling neuron output
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

### Readout Layer

In [15]:
# Classify the output into 10 classes
# Define the weight and biases
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

In [16]:
# Then we calculate the output
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

### Training and Evaluation of the model

In [17]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [19]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(20000):
        batch = mnist.train.next_batch(50)
        if i % 100 == 0:
            train_accuracy = accuracy.eval(feed_dict={
              x: batch[0], y_: batch[1], keep_prob: 1.0})
            print('step %d, training accuracy %g' % (i, train_accuracy))
        train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

        print('test accuracy %g' % accuracy.eval(feed_dict={
          x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.14
test accuracy 0.1109
test accuracy 0.094
test accuracy 0.1303
test accuracy 0.1468
test accuracy 0.1553
test accuracy 0.1556
test accuracy 0.1492
test accuracy 0.1516
test accuracy 0.1563
test accuracy 0.1631
test accuracy 0.1741
test accuracy 0.1909
test accuracy 0.2146
test accuracy 0.2535
test accuracy 0.2922
test accuracy 0.3356
test accuracy 0.3747
test accuracy 0.4059
test accuracy 0.416
test accuracy 0.4274
test accuracy 0.4374
test accuracy 0.4559
test accuracy 0.4774
test accuracy 0.4884
test accuracy 0.4894
test accuracy 0.4876
test accuracy 0.485
test accuracy 0.4929
test accuracy 0.4993
test accuracy 0.5181
test accuracy 0.5343
test accuracy 0.5537
test accuracy 0.5699
test accuracy 0.5869
test accuracy 0.5955
test accuracy 0.6123
test accuracy 0.6273
test accuracy 0.6503
test accuracy 0.6688
test accuracy 0.6806
test accuracy 0.6857
test accuracy 0.6901
test accuracy 0.6948
test accuracy 0.6933
test accuracy 0.695
test accuracy 0.7017
test ac

KeyboardInterrupt: 